In [8]:
# 1. Import library
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split
import pandas as pd
import warnings

In [9]:
warnings.filterwarnings("ignore", message=".*load_learner.*insecure pickle.*")

# 2. Load the data
train_df = pd.read_csv('../Data/processed/0427_01/train_2025.csv') 
test_df = pd.read_csv('../Data/processed/0427_01/test_2025.csv') 

train_df.drop(columns="claim_number", inplace=True)
test_id = test_df['claim_number']
test_df.drop(columns=["claim_number"], inplace=True)

In [10]:
predictor = TabularPredictor(
    label="fraud",
    eval_metric="f1",
    problem_type="binary"
).fit(
    train_data=train_df,
    holdout_frac=0.2,
    presets="best",
    verbosity=1
)

No path specified. Models will be saved in: "AutogluonModels\ag-20250428_035409"
Preset alias specified: 'best' maps to 'best_quality'.


KeyboardInterrupt: 

In [13]:
predictor.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.354347,f1,13.738319,1263.271420,0.003999,3.735987,3,True,77
1,NeuralNetFastAI_BAG_L2,0.340444,f1,12.535254,1144.790274,0.385176,21.977843,2,True,58
2,NeuralNetFastAI_r102_BAG_L2,0.330823,f1,12.354052,1132.691576,0.203974,9.879144,2,True,72
3,WeightedEnsemble_L2,0.329344,f1,1.888623,164.877765,0.003509,4.035510,2,True,50
4,NeuralNetFastAI_r191_BAG_L2,0.327439,f1,12.802109,1171.076867,0.652030,48.264436,2,True,65
...,...,...,...,...,...,...,...,...,...,...
72,RandomForestGini_BAG_L1,0.002795,f1,0.498071,1.113853,0.498071,1.113853,1,True,5
73,LightGBM_r96_BAG_L1,0.000000,f1,0.029065,1.624056,0.029065,1.624056,1,True,19
74,LightGBM_r196_BAG_L1,0.000000,f1,0.029551,2.669791,0.029551,2.669791,1,True,44
75,XGBoost_r98_BAG_L1,0.000000,f1,0.096112,2.404294,0.096112,2.404294,1,True,49


In [14]:
# 4. Predict on the test set
predictions = predictor.predict(test_df)

test_df = pd.read_csv('Data/original/test_2025.csv')
# 5. Save predictions to CSV
submission = pd.DataFrame({
    "claim_number": test_id,  # Important: use the original claim_number
    "fraud": predictions                      # Your predicted fraud labels (0 or 1)
})
submission.to_csv("submissions/submission.csv", index=False)